In [ ]:
import pandas as pd

if pd.read_csv('../input/siim-covid19-detection/sample_submission.csv').shape[0] != 2477:
    
    !bash ../input/preprocessing/setup.sh
    import os
    import cv2

    from PIL import Image
    from tqdm.auto import tqdm

    import numpy as np
    import pandas as pd
    import pydicom
    from pydicom.pixel_data_handlers.util import apply_voi_lut

    import torch
    import torch.nn as nn
    import torch.nn.functional as F
    from torch.utils.data import Dataset,DataLoader,ConcatDataset
    import albumentations as A
    from albumentations.pytorch import ToTensorV2

    from sklearn.metrics import average_precision_score
    from glob import glob
    import shutil
    import matplotlib.pyplot as plt
    from sklearn.model_selection import GroupKFold
    import seaborn as sns
    import pandas as pd
    import numpy as np
    import gc

    from PIL import Image
    from tqdm.auto import tqdm

    # pytorch lighting
    import pytorch_lightning as pl
    import pickle
    import warnings 
    import sys
    sys.path.append('../input/timm210703/pytorch-image-models-master-210703')
    import timm

    import torch.nn.functional as F

    import torch
    import torch.nn as nn

    warnings.filterwarnings('ignore')
    !python ../input/preprocessing/preprocess.py --img_size 640 --output_dir /kaggle/temp/

    def compute_oof_map(df):
        negative_map = average_precision_score(df['Negative for Pneumonia'], df['negative'])
        typical_map = average_precision_score(df['Typical Appearance'], df['typical'])
        indeterminate_map = average_precision_score(df['Indeterminate Appearance'], df['indeterminate'])
        atypical_map = average_precision_score(df['Atypical Appearance'], df['atypical'])
        map = (negative_map + typical_map + indeterminate_map + atypical_map) / 6

        return map

    def memory_cleanup():
        """
        Cleans up GPU memory
        https://github.com/huggingface/transformers/issues/1742
        """
        print('Cleaning GPU....')
        for obj in gc.get_objects():
            if torch.is_tensor(obj):
                del obj

        gc.collect()
        torch.cuda.empty_cache()

    root_dir1 = '../input/siim-covid19-pred-oofs/siim-covid19-pred-oofs/'
    model_0_path = f'{root_dir1}tf_efficientnet_b5_ns/'
    model_1_path = f'{root_dir1}tf_efficientnet_b7_ns/'
    model_2_path = f'{root_dir1}tf_efficientnetv2_m/'
    model_3_path = f'{root_dir1}swin_base_patch4_window12_384/'
    model_4_path = f'../input/effnet-b5-models-test-2477-pseudo/tf_efficientnet_b5_ns/'
#     model_1_path = f'../input/effnet-b5-models-v3/tf_efficientnet_b5_ns/'
#     model_2_path = f'../input/effnet-b7-models-v3/tf_efficientnet_b7_ns/'
#     model_3_path = f'../input/effnetv2-m-models-v3/tf_efficientnetv2_m/'
#     model_4_path = f'../input/swim-trans-models-v3/'

    models_states = [
        [
            f'{model_0_path}/fold_0/oof_preds_tta_flip.csv',
            f'{model_0_path}/fold_1/oof_preds_tta_flip.csv',
            f'{model_0_path}/fold_2/oof_preds_tta_flip.csv',
            f'{model_0_path}/fold_3/oof_preds_tta_flip.csv',
            f'{model_0_path}/fold_4/oof_preds_tta_flip.csv',
        ],
        [
            f'{model_1_path}/fold_0/oof_preds_tta_flip.csv',
            f'{model_1_path}/fold_1/oof_preds_tta_flip.csv',
            f'{model_1_path}/fold_2/oof_preds_tta_flip.csv',
            f'{model_1_path}/fold_3/oof_preds_tta_flip.csv',
            f'{model_1_path}/fold_4/oof_preds_tta_flip.csv',
        ],
        [
            f'{model_2_path}/fold_0/oof_preds_tta_flip.csv',
            f'{model_2_path}/fold_1/oof_preds_tta_flip.csv',
            f'{model_2_path}/fold_2/oof_preds_tta_flip.csv',
            f'{model_2_path}/fold_3/oof_preds_tta_flip.csv',
            f'{model_2_path}/fold_4/oof_preds_tta_flip.csv',
        ],
        [
            f'{model_3_path}/fold_0/oof_preds_tta_flip.csv',
            f'{model_3_path}/fold_1/oof_preds_tta_flip.csv',
            f'{model_3_path}/fold_2/oof_preds_tta_flip.csv',
            f'{model_3_path}/fold_3/oof_preds_tta_flip.csv',
            f'{model_3_path}/fold_4/oof_preds_tta_flip.csv',
        ],
        [
            f'{model_4_path}/fold_0/oof_preds_tta_flip.csv',
            f'{model_4_path}/fold_1/oof_preds_tta_flip.csv',
            f'{model_4_path}/fold_2/oof_preds_tta_flip.csv',
            f'{model_4_path}/fold_3/oof_preds_tta_flip.csv',
            f'{model_4_path}/fold_4/oof_preds_tta_flip.csv',
        ],
    ]

    study_folds_csv_path = '../input/train-study-folds/train_study_folds_washen_v2.csv'
    study_folds = pd.read_csv(study_folds_csv_path)

    study_folds = study_folds[['image','None','Opacity','fold']]
    study_folds.columns = ['image','None','Opacity','fold_n']

    z = [['Atypical Appearance','atypical'],
        ['Negative for Pneumonia','negative'],
        ['Indeterminate Appearance','indeterminate'],
        ['Typical Appearance','typical'],
        ['None','negative']
        ]

    ww = []
    tt = []
    final_score = []
    for jj in z:
        t,p = jj[0], jj[1]
        oof_zoo = []
        for index, states in enumerate(models_states):
            oof = []
            for fold,state in enumerate(states):
                none_df = study_folds[study_folds.fold_n == fold]
                df = pd.read_csv(state)
                if index == 4:
                    df.columns = df.columns.str.replace('None', 'None_bimcv_p')
                    df.columns = df.columns.str.replace('Opacity', 'Opacity_bimcv_p')
                df = pd.merge(df, none_df, on = 'image', how = 'left')


                oof.append(df.loc[:,['study_id',t,p, 'fold']])
            oof_zoo.append(pd.concat(oof).reset_index())

        x = np.zeros(( len(oof_zoo[0]),len(oof_zoo) ))
        for k in range(len(oof_zoo)):
            x[:,k] = oof_zoo[k][p].values

        TRUE = oof_zoo[0][t].values
        all = []
        for k in range(x.shape[1]):
            auc = average_precision_score(oof_zoo[0][t],x[:,k])
            all.append(auc)
            print('Model %i has OOF AUC = %.4f'%(k,auc))

        m = [np.argmax(all)]; w = []
        old = np.max(all); 

        RES = 100; 
        PATIENCE = 30; 
        TOL = 0.0000003
        DUPLICATES = True

        print('Ensemble AUC = %.4f by beginning with model %i'%(old,m[0]))
        print()

        for kk in range(len(oof_zoo)):

            # BUILD CURRENT ENSEMBLE
            md = x[:,m[0]]
            for i,k in enumerate(m[1:]):
                md = w[i]*x[:,k] + (1-w[i])*md

            # FIND MODEL TO ADD
            mx = 0; mx_k = 0; mx_w = 0
            print('Searching for best model to add... ')

            # TRY ADDING EACH MODEL
            for k in range(x.shape[1]):
                print(k,', ',end='')
                if not DUPLICATES and (k in m): continue

                # EVALUATE ADDING MODEL K WITH WEIGHTS W
                bst_j = 0; bst = 0; ct = 0
                for j in range(RES):
                    tmp = j/RES*x[:,k] + (1-j/RES)*md
                    auc =  average_precision_score(TRUE,tmp**0.5)
                    if auc>bst:
                        bst = auc
                        bst_j = j/RES
                    else: ct += 1
                    if ct>PATIENCE: continue
                if bst>mx:
                    mx = bst
                    mx_k = k
                    mx_w = bst_j

            # STOP IF INCREASE IS LESS THAN TOL
            inc = mx-old
            if inc<=TOL: 
                print(); print('No increase. Stopping.')
                continue

            # DISPLAY RESULTS
            print(); #print(kk,mx,mx_k,mx_w,'%.5f'%inc)
            print('Ensemble AUC = %.4f after adding model %i with weight %.3f. Increase of %.4f'%(mx,mx_k,mx_w,inc))
            print()

            old = mx; m.append(mx_k); w.append(mx_w)
        print('We are using models',m)
        ww.append(w)
        tt.append(m)
        print('with weights',w)
        print('and achieve ensemble AUC = %.4f'%old)
        final_score.append(old)
        md = x[:,m[0]]
        for i,k in enumerate(m[1:]):
            md = w[i]*x[:,k] + (1-w[i])*md
        df = oof_zoo[0].copy()
        df[p] = md
    #     df.to_csv(f'ensemble_oof_{p}.csv',index=False)

    print(f'Total mAP: {sum(final_score)/6}')

    study2images_d = pickle.load(open('/kaggle/temp/study2images_d.pickle', 'rb'))
    sub_df = pd.read_csv('../input/siim-covid19-detection/sample_submission.csv')
    image2shape_d = pd.read_pickle('/kaggle/temp/image2shape_d.pickle')
    image_df = sub_df.copy()

    class Config:
        data_dir = '../input/siim-covid19-detection/'
        test_csv_path = data_dir + 'sample_submission.csv'

        num_classes = 4
        model_name = 'tf_efficientnet_b5_ns'
    #     model_name = 'tf_efficientnet_b7_ns'
    #     model_name = 'tf_efficientnetv2_m'
        image_size = 512            ### input size in training

    #     model_name = 'swin_base_patch4_window12_384'
    #     image_size = 384            ### input size in training
        batch_size = 32
        num_workers = 4

        device = 'cuda'             ### set gpu or cpu mode
        debug = False              ### debug flag for checking your modify code

    CFG = Config

    test_df = pd.read_csv(CFG.test_csv_path)
    study_len = test_df[test_df.id.str.contains('study')].index
    test_df = test_df[test_df.id.str.contains('image')]

    test_df['negative'] = 0
    test_df['typical'] = 0
    test_df['indeterminate'] = 0
    test_df['atypical'] = 0

    label_cols = test_df.columns[2:]

    def to_tensor(x, **kwargs):
        if x.ndim==2 : 
            x = np.expand_dims(x,2)
        x = np.transpose(x,(2,0,1)).astype('float32') / 255.

        x = torch.from_numpy(x)
        return x

    def get_preprocessing():
        """Construct preprocessing transform

        Args:
            preprocessing_fn (callbale): data normalization function 
                (can be specific for each pretrained neural network)
        Return:
            transform: albumentations.Compose

        """

        _transform = [
    #         A.Lambda(image=preprocessing_fn),
            A.Lambda(image=to_tensor, mask=to_tensor),
        ]
        return A.Compose(_transform)


    def get_val_transforms(CFG):
        return A.Compose([
                A.Resize(CFG.image_size, CFG.image_size),
    #             A.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225], max_pixel_value=255.0, p=1.0),
    #             ToTensorV2(),
            ],p=1.0)

    class SIIMTestDataset(Dataset):
        def __init__(self, df, transforms=None, preprocessing=None):
            super().__init__()
            self.df = df
            self.transforms = transforms
            self.preprocessing = preprocessing
            self.length = len(df)

        def __len__(self):
            return self.length

        def __getitem__(self, index):
            d = self.df.iloc[index]
            split = d.id.split('_')[-1]
            image_path =  '/kaggle/temp/test/%s.png' % (d.id[:-6])
            image = cv2.imread(image_path)
            image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
            if image is None:
                raise FileNotFoundError(image_path)

            # apply augmentations
            if self.transforms:
                image = self.transforms(image=image)['image']
            else:
                image = torch.from_numpy(image)

            # apply preprocessing
            if self.preprocessing:
                sample = self.preprocessing(image=image)
                image = sample['image']

            return image


    def gem(x, p=3, eps=1e-6):
        return F.avg_pool2d(x.clamp(min=eps).pow(p), (x.size(-2), x.size(-1))).pow(1.0 / p)

    class GeM(nn.Module):
        def __init__(self, p=3, eps=1e-6):
            super(GeM, self).__init__()
            self.p = nn.Parameter(torch.ones(1) * p)
            self.eps = eps
        def forward(self, x):
            return gem(x, p=self.p, eps=self.eps)[:,:,0,0]
        def __repr__(self):
            return (
                self.__class__.__name__
                + f"(p={self.p.data.tolist()[0]:.4f}, eps={str(self.eps)})"
            )

    class SIIMMaskNet(nn.Module):
        def __init__(self, model_name, num_classes, pretrained=False):
            super(SIIMMaskNet,self).__init__()
            self.model_name = model_name
            self.model = timm.create_model(model_name, pretrained=pretrained)

            ### effnet
            if model_name == 'tf_efficientnet_b5_ns' or model_name == 'tf_efficientnet_b7_ns':
    #             self.model.global_pool = GeM()
                num_features = self.model.classifier.in_features  
                self.model.classifier = nn.Linear(num_features, num_classes)

            if model_name == 'tf_efficientnetv2_m':   
                num_features = self.model.classifier.in_features  
                self.model.classifier = nn.Linear(num_features, num_classes)

            if model_name == 'swin_base_patch4_window12_384':
                num_features = self.model.head.in_features  
                self.model.head = nn.Linear(num_features, num_classes) 


            hidden_dims = 128
            ## for effnet 
            if model_name == 'tf_efficientnet_b5_ns' or model_name == 'tf_efficientnetv2_m':
                self.mask = nn.Sequential(
                    nn.Conv2d(176, hidden_dims, kernel_size=3, padding=1), 
                    nn.BatchNorm2d(hidden_dims),
                    nn.ReLU(inplace=True),
                    nn.Conv2d(hidden_dims, hidden_dims, kernel_size=3, padding=1),
                    nn.BatchNorm2d(hidden_dims),
                    nn.ReLU(inplace=True),
                    nn.Conv2d(hidden_dims, 1, kernel_size=1, padding=0),
                )

            if model_name == 'tf_efficientnet_b7_ns':
                self.mask = nn.Sequential(
                    nn.Conv2d(224, hidden_dims, kernel_size=3, padding=1), 
                    nn.BatchNorm2d(hidden_dims),
                    nn.ReLU(inplace=True),
                    nn.Conv2d(hidden_dims, hidden_dims, kernel_size=3, padding=1),
                    nn.BatchNorm2d(hidden_dims),
                    nn.ReLU(inplace=True),
                    nn.Conv2d(hidden_dims, 1, kernel_size=1, padding=0),
                )    

            ### for transformer
            if model_name == 'swin_base_patch4_window12_384':
                self.mask = nn.Sequential(
                    nn.Conv2d(512, hidden_dims, kernel_size=3, padding=1), ### swin
                    nn.BatchNorm2d(hidden_dims),
                    nn.ReLU(inplace=True),
                    nn.Conv2d(hidden_dims, hidden_dims, kernel_size=3, padding=1),
                    nn.BatchNorm2d(hidden_dims),
                    nn.ReLU(inplace=True),
                    nn.Conv2d(hidden_dims, 1, kernel_size=1, padding=0),
                )


        def forward(self, x):
            if self.model_name == 'tf_efficientnet_b5_ns' or self.model_name == 'tf_efficientnet_b7_ns' or self.model_name == 'tf_efficientnetv2_m':
                x = self.model.act1(self.model.bn1(self.model.conv_stem(x)))
                x = self.model.blocks[0](x)
                x = self.model.blocks[1](x)
                x = self.model.blocks[2](x)
                x = self.model.blocks[3](x)
                x = self.model.blocks[4](x)

                ####----
                mask = self.mask(x)
                ####----

                x = self.model.blocks[5](x)
                x = self.model.blocks[6](x)
                conv5 = self.model.act2(self.model.bn2(self.model.conv_head(x)))

                feat = self.model.global_pool(conv5).view(conv5.size(0), -1)
                logits = self.model.classifier(feat)

            if self.model_name == 'swin_base_patch4_window12_384':
                x = self.model.patch_embed(x)
                if self.model.absolute_pos_embed is not None:
                    x = x + self.model.absolute_pos_embed
                x = self.model.pos_drop(x)
                x = self.model.layers[0](x)
                x = self.model.layers[1](x)

    #             x = self.model.layers[2](x)
                for blk in self.model.layers[2].blocks:
                    if not torch.jit.is_scripting() and self.model.layers[2].use_checkpoint:
                        x = checkpoint.checkpoint(blk, x)
                    else:
                        x = blk(x)

                ####----
                # B L C to B C L
                x_mask = x.transpose(1, 2)
                # B C L to B C H W
                x_mask = x_mask.reshape(x_mask.size(0),x_mask.size(1), 24, 24) # 24 = 384 / 16
                mask = self.mask(x_mask)
                ####----

                if self.model.layers[2].downsample is not None:
                    x = self.model.layers[2].downsample(x)

                x = self.model.layers[3](x)
                conv5 = x
                x = self.model.norm(x)  # B L C
                x = self.model.avgpool(x.transpose(1, 2))  # B C 1
                feat = torch.flatten(x, 1)
                logits = self.model.head(feat)

            return logits, mask

    class SIIMPLModel(pl.LightningModule):
        def __init__(self):
            super(SIIMPLModel,self).__init__()
            self.model = SIIMMaskNet(CFG.model_name, CFG.num_classes, pretrained=False)

        def forward(self, x):
            return self.model(x)

    def do_predict(model, test_loader, tta=['']):
        print(f'tta is {tta}')
        test_probability = []
        test_num = 0
        tk0 = tqdm(enumerate(test_loader), total=len(test_loader))
        for t, (image) in tk0:
            batch_size = image.size(0)
            image = image.to(device)
            #<todo> TTA
            model.eval()
            with torch.no_grad():
                probability = []
                logit, mask = model(image)
                probability.append(F.softmax(logit,-1))

                if 'flip' in tta:
                    logit, mask = model(torch.flip(image,dims=(3,)))
                    probability.append(F.softmax(logit,-1))

                if 'scale' in tta:
                    # size=None, scale_factor=None, mode='nearest', align_corners=None, recompute_scale_factor=None):
                    logit, mask = model(F.interpolate(image, scale_factor=1.25, mode='bilinear', align_corners=False))
                    probability.append(F.softmax(logit,-1))

                #--------------
                probability = torch.stack(probability,0).mean(0)

            test_num += batch_size
            test_probability.append(probability.data.cpu().numpy())
        assert(test_num == len(test_loader.dataset))

        probability = np.concatenate(test_probability)
        return probability

    # ====================================================
    # Helper functions
    # ====================================================
    def inference(model, states, test_loader, device, ttas):
        probs = 0
        for (state,tta) in zip(states,ttas):
            model = model.load_from_checkpoint(state)
            model.to(device)
            probability = do_predict(model, test_loader, tta)
            probs += probability**0.5
        probs = probs/len(states) 
        return probs

    # ====================================================
    # inference
    # ====================================================
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

    SUB_CSV = []

    ### load test dataloader
    CFG.image_size = 512
    test_dataset_512 = SIIMTestDataset(test_df, transforms=get_val_transforms(CFG), preprocessing=get_preprocessing())
    test_loader_512 = DataLoader(test_dataset_512, CFG.batch_size, num_workers=CFG.num_workers, shuffle=False) 

    CFG.image_size = 384
    test_dataset_384 = SIIMTestDataset(test_df, transforms=get_val_transforms(CFG), preprocessing=get_preprocessing())
    test_loader_384 = DataLoader(test_dataset_384, CFG.batch_size, num_workers=CFG.num_workers, shuffle=False) 
    memory_cleanup()
    
    
    #### b5 v1
    CFG.model_name = 'tf_efficientnet_b5_ns'
    CFG.image_size = 512

    ### init pl model
    pl_model = SIIMPLModel()

    ### load test dataloader
    if CFG.image_size == 512:
        test_loader = test_loader_512
    else:
        test_loader = test_loader_384

    model_path = '../input/efficientnetb5models/tf_efficientnet_b5_ns/'

    states = [
        f'{model_path}/fold0_best_mAP.ckpt',
        f'{model_path}/fold1_best_mAP.ckpt',
        f'{model_path}/fold2_best_mAP.ckpt',
        f'{model_path}/fold3_best_mAP.ckpt',
        f'{model_path}/fold4_best_mAP.ckpt',
    ]

    fold_ttas = [
        ['flip'],
        ['flip'],
        ['flip'],
        ['flip'],
        ['flip'],
        ]
    predictions = inference(pl_model, states, test_loader, device, fold_ttas)

    pred_df = test_df.copy()
    pred_df[label_cols] = predictions
    print(pred_df.head())
    SUB_CSV.append(pred_df)
    # SUB_CSV
    memory_cleanup()
    
    #### b7 v1
    CFG.model_name = 'tf_efficientnet_b7_ns'
    CFG.image_size = 512

    ### init pl model
    pl_model = SIIMPLModel()

    ### load test dataloader
    if CFG.image_size == 512:
        test_loader = test_loader_512
    else:
        test_loader = test_loader_384

    model_path = '../input/efficientnetb7models/tf_efficientnet_b7_ns/'

    states = [
        f'{model_path}/fold0_best_mAP.ckpt',
        f'{model_path}/fold1_best_mAP.ckpt',
        f'{model_path}/fold2_best_mAP.ckpt',
        f'{model_path}/fold3_best_mAP.ckpt',
        f'{model_path}/fold4_best_mAP.ckpt',
    ]

    fold_ttas = [
        ['flip'],
        ['flip'],
        ['flip'],
        ['flip'],
        ['flip'],
        ]
    predictions = inference(pl_model, states, test_loader, device, fold_ttas)

    pred_df = test_df.copy()
    pred_df[label_cols] = predictions
    print(pred_df.head())
    SUB_CSV.append(pred_df)
    # SUB_CSV
    memory_cleanup()

    #### v2m v1
    CFG.model_name = 'tf_efficientnetv2_m'
    CFG.image_size = 512
    ### init pl model
    pl_model = SIIMPLModel()

    ### load test dataloader
    if CFG.image_size == 512:
        test_loader = test_loader_512
    else:
        test_loader = test_loader_384

    model_path = '../input/efficientnetv2mmodels/tf_efficientnetv2_m/'

    states = [
        f'{model_path}/fold0_best_mAP.ckpt',
        f'{model_path}/fold1_best_mAP.ckpt',
        f'{model_path}/fold2_best_mAP.ckpt',
        f'{model_path}/fold3_best_mAP.ckpt',
        f'{model_path}/fold4_best_mAP.ckpt',
    ]

    fold_ttas = [
        ['flip'],
        ['flip'],
        ['flip'],
        ['flip'],
        ['flip'],
        ]
    predictions = inference(pl_model, states, test_loader, device, fold_ttas)

    pred_df = test_df.copy()
    pred_df[label_cols] = predictions
    print(pred_df.head())
    SUB_CSV.append(pred_df)
    # SUB_CSV
    memory_cleanup()

    #### swim-trans v1
    CFG.model_name = 'swin_base_patch4_window12_384'
    CFG.image_size = 384
    ### init pl model
    pl_model = SIIMPLModel()

    ### load test dataloader
    if CFG.image_size == 512:
        test_loader = test_loader_512
    else:
        test_loader = test_loader_384

    model_path = '../input/swimtrans384models/swin_base_patch4_window12_384/'

    states = [
        f'{model_path}/fold0_best_mAP.ckpt',
        f'{model_path}/fold1_best_mAP.ckpt',
        f'{model_path}/fold2_best_mAP.ckpt',
        f'{model_path}/fold3_best_mAP.ckpt',
        f'{model_path}/fold4_best_mAP.ckpt',
    ]

    fold_ttas = [
        ['flip'],
        ['flip'],
        ['flip'],
        ['flip'],
        ['flip'],
        ]
    predictions = inference(pl_model, states, test_loader, device, fold_ttas)

    pred_df = test_df.copy()
    pred_df[label_cols] = predictions
    print(pred_df.head())
    SUB_CSV.append(pred_df)
    # SUB_CSV
    memory_cleanup()
    
    
     #### b5 test pseudo
    CFG.model_name = 'tf_efficientnet_b5_ns'
    CFG.image_size = 512

    ### init pl model
    pl_model = SIIMPLModel()

    ### load test dataloader
    if CFG.image_size == 512:
        test_loader = test_loader_512
    else:
        test_loader = test_loader_384

    model_path = '../input/effnet-b5-models-test-2477-pseudo/tf_efficientnet_b5_ns/'

    states = [
        f'{model_path}/fold0_best_mAP.ckpt',
        f'{model_path}/fold1_best_mAP.ckpt',
        f'{model_path}/fold2_best_mAP.ckpt',
        f'{model_path}/fold3_best_mAP.ckpt',
        f'{model_path}/fold4_best_mAP.ckpt',
    ]

    fold_ttas = [
        ['flip'],
        ['flip'],
        ['flip'],
        ['flip'],
        ['flip'],
        ]
    predictions = inference(pl_model, states, test_loader, device, fold_ttas)

    pred_df = test_df.copy()
    pred_df[label_cols] = predictions
    print(pred_df.head())
    SUB_CSV.append(pred_df)
    # SUB_CSV
    memory_cleanup()

    test_df = SUB_CSV[0].copy()
    for ix,jj in enumerate(z):
        t,p = jj[0], jj[1]
        y = np.zeros(( len(SUB_CSV[0]),len(SUB_CSV) ))
        for k in range(len(SUB_CSV)):
            y[:,k] = SUB_CSV[k][p].values

        md2 = y[:,tt[ix][0]]
        for i,k in enumerate(tt[ix][1:]):
            md2 = ww[ix][i]*y[:,k] + (1-ww[ix][i])*md2
        test_df[p] = md2**0.5

    index = sub_df.index
    sub_df[label_cols] = 0.0
    for i,j in tqdm(study2images_d.items()):
        x = []
        for k in j:
            temp = test_df[test_df['id'] == (k+'_image')]
            x.append(np.mean(temp[label_cols].values, axis = 0))
        condition = sub_df["id"] == i+'_study'
        ndices = index[condition]
        sub_df.loc[ndices,label_cols] = np.mean(np.array(x), axis = 0)

    pd.concat([sub_df,test_df], axis = 0).to_csv('pred_study.csv', index = False)

    for i in range(max(study_len)+1):
        x = sub_df.iloc[i,:].values
        sub_df.loc[i,'PredictionString'] = f'negative {x[2]} 0 0 1 1 typical {x[3]} 0 0 1 1 indeterminate {x[4]} 0 0 1 1 atypical {x[5]} 0 0 1 1'

    sub_df = sub_df[['id', 'PredictionString']]

    sub_df.to_csv('submission_study.csv', index = False)

    %cd ../input/siimyolov5develop/yolov5-develop/

    test_dir = f'/kaggle/temp/test'
    image_size = 384
    conf_thres = 0.0001
    iou_thres = 0.5
    infer_fold = [0,1,2,3,4]

#     models_dir = '/kaggle/input/yolov5384foldsmodels/yolov5-models'
    models_dir = '/kaggle/input/yolotrs384pseudo/yolotrs-384-test-pseudo'


    shutil.copytree('/kaggle/input/siimyolov5develop/yolov5-develop', '/kaggle/working/yolov5-develop')
    os.chdir('/kaggle/working/yolov5-develop')

    from glob import glob

    def yolo2voc(image_height, image_width, bboxes):
        """
        yolo => [xmid, ymid, w, h] (normalized)
        voc  => [x1, y1, x2, y1]

        """ 
        bboxes = bboxes.copy().astype(float) # otherwise all value will be 0 as voc_pascal dtype is np.int

        bboxes[..., [0, 2]] = bboxes[..., [0, 2]]* image_width
        bboxes[..., [1, 3]] = bboxes[..., [1, 3]]* image_height

        bboxes[..., [0, 1]] = bboxes[..., [0, 1]] - bboxes[..., [2, 3]]/2
        bboxes[..., [2, 3]] = bboxes[..., [0, 1]] + bboxes[..., [2, 3]]

        return bboxes


    for fold in infer_fold:
        weight_path = f'{models_dir}/fold{fold}_best.pt'
        save_name = f'exp-siim-yolotrs-384-1cls-fold{fold}'
#         weight_0_path = f'{models_dir}/yolotrs-384/fold{fold}_best.pt'
#         weight_1_path = f'{models_dir}/yolotrl-384/fold{fold}_best.pt'
#         weight_2_path = f'{models_dir}/yolov5x-384/fold{fold}_best.pt'
#         save_name = f'exp-siim-ensemble3-384-1cls-fold{fold}'
        !python detect.py --weights $weight_path\
                          --source $test_dir\
                          --img {image_size} \
                          --conf {conf_thres} \
                          --iou-thres {iou_thres} \
                          --save-txt \
                          --save-conf \
                          --augment \
                          --name $save_name

        PRED_PATH = f'runs/detect/{save_name}/labels'
        prediction_files = os.listdir(PRED_PATH)
        print('Number of test images predicted as opaque: ', len(prediction_files))


        memory_cleanup()
        PRED_PATH = f'runs/detect/{save_name}/labels'

        image_ids = []
        PredictionStrings = []
        for file_path in tqdm(glob(f'{PRED_PATH}/*.txt')):
            image_id = file_path.split('/')[-1].split('.')[0]
            h, w = image2shape_d[image_id]
            f = open(file_path, 'r')
            data = np.array(f.read().replace('\n', ' ').strip().split(' ')).astype(np.float32).reshape(-1, 6)
            data = data[:, [0, 5, 1, 2, 3, 4]]
            bboxes = list(np.round(np.concatenate((data[:, :2], np.round(yolo2voc(h, w, data[:, 2:]))), axis =1).reshape(-1), 12).astype(str))

            for idx in range(len(bboxes)):
                bboxes[idx] = str(int(float(bboxes[idx]))) if idx%6!=1 else bboxes[idx]
            image_ids.append(image_id+'_image')
            PredictionStrings.append(' '.join(bboxes))

        pred_df = pd.DataFrame({'id':image_ids,
                                'PredictionString':PredictionStrings})

    #     print(pred_df.head())
        temp_image_df = sub_df.copy()
        temp_image_df = temp_image_df.drop(['PredictionString'], axis=1)
        sub_df = pd.merge(temp_image_df, pred_df, on = 'id', how = 'left').fillna("none 1 0 0 1 1")
        sub_df = sub_df[['id', 'PredictionString']]
        for i in range(sub_df.shape[0]):
            if sub_df.loc[i,'PredictionString'] == "none 1 0 0 1 1":
                continue
            sub_df_split = sub_df.loc[i,'PredictionString'].split()
            sub_df_list = []
            for j in range(int(len(sub_df_split) / 6)):
                sub_df_list.append('opacity')
                sub_df_list.append(sub_df_split[6 * j + 1])
                sub_df_list.append(sub_df_split[6 * j + 2])
                sub_df_list.append(sub_df_split[6 * j + 3])
                sub_df_list.append(sub_df_split[6 * j + 4])
                sub_df_list.append(sub_df_split[6 * j + 5])
            sub_df.loc[i,'PredictionString'] = ' '.join(sub_df_list)
        #### Paths to 5-fold yolotrs csv
        sub_df.to_csv(f'/kaggle/working/ensemble3_image_fold{fold}.csv',index=False)

    sample_df = pd.read_csv('/kaggle/input/siim-covid19-detection/sample_submission.csv')
    ## load study part
    study_pred_df = pd.read_csv('/kaggle/working/pred_study.csv')
    study_image_df = study_pred_df[sample_df.shape[0]:]
    study_part_df = study_pred_df[:len(study_len)]
    study_df = pd.concat([study_part_df,study_image_df], axis = 0).reset_index(drop=True) ### this 2477 images predict results about study part
    study_df.columns = ['id','PredictionString_study','negative','typical','indeterminate','atypical']
    ### merge study
    sample_df = pd.merge(sample_df,study_df,on='id',how='left')
#     sample_df

    ### load image part with fold 1
    image_df = pd.read_csv('/kaggle/working/ensemble3_image_fold1.csv')
    image_df.columns = ['id','PredictionString_image']
    ### merge image part
    sample_df = pd.merge(sample_df,image_df,on='id',how='left')
    
    for i in range(len(sample_df)):
        if i < len(study_len):
            negative = sample_df.loc[i,'negative']
            typical = sample_df.loc[i,'typical']
            indeterminate = sample_df.loc[i,'indeterminate']
            atypical = sample_df.loc[i,'atypical']
            sample_df.loc[i,'PredictionString'] = f'negative {negative} 0 0 1 1 typical {typical} 0 0 1 1 indeterminate {indeterminate} 0 0 1 1 atypical {atypical} 0 0 1 1'
        else:
            negative = sample_df.loc[i,'negative']
            if 'none' in sample_df.loc[i,'PredictionString_image']:
                sample_df.loc[i,'PredictionString'] = f'none {negative} 0 0 1 1'
            else:
                image_preds = sample_df.loc[i,'PredictionString_image']
                sample_df.loc[i,'PredictionString'] = f'none {negative} 0 0 1 1 {image_preds}'

    sample_df = sample_df[['id','PredictionString']].reset_index(drop=True)
    #### path for study part
    sample_df.to_csv('/kaggle/working/study_2477_images.csv', index = False)

    %cd /kaggle/working
    !mkdir -p /kaggle/temp/hung/image-level
    !cp -ar /kaggle/input/image-level-submission-code-siim/* /kaggle/temp/hung/image-level
    !mkdir -p /kaggle/temp/hung/post-process
    !cp -ar /kaggle/input/post-processing-submission-code-siim/* /kaggle/temp/hung/post-process

    # %%capture cell_print
    %cd /kaggle/temp/hung/image-level
    !python run.py -checkpoints \
    /kaggle/input/yolov5-checkpoints-siim/best.pt \
    /kaggle/input/yolov5-checkpoints-siim/best1.pt \
    /kaggle/input/yolov5-checkpoints-siim/best2.pt \
    /kaggle/input/yolov5-checkpoints-siim/best3.pt \
    /kaggle/input/yolov5-checkpoints-siim/best4.pt \
    -image-dir /kaggle/temp/test \
    -save-dir /kaggle/working/image_sub \
    -img2shape /kaggle/temp/image2shape_d.pickle \
    --image-size 640 \
    --batch 256 \
    --conf 0.0001
    # %%capture cell_print
    %cd /kaggle/temp/hung/image-level
    !python run.py -checkpoints \
    /kaggle/input/hung-yolo/best_fold_0.pt \
    /kaggle/input/hung-yolo/best_fold_1.pt \
    /kaggle/input/hung-yolo/best_fold_2.pt \
    /kaggle/input/hung-yolo/best_fold_3.pt \
    /kaggle/input/hung-yolo/best_fold_4.pt \
    -image-dir /kaggle/temp/test \
    -save-dir /kaggle/working/image_sub \
    -img2shape /kaggle/temp/image2shape_d.pickle \
    --image-size 512 \
    --batch 256 \
    --conf 0.0001

    %cd /kaggle/temp/hung/post-process
    !python run.py \
    -study-csv \
    /kaggle/working/study_2477_images.csv \
    -image-csv \
    /kaggle/working/image_sub/best.csv \
    /kaggle/working/image_sub/best1.csv \
    /kaggle/working/image_sub/best2.csv \
    /kaggle/working/image_sub/best3.csv \
    /kaggle/working/image_sub/best4.csv \
    /kaggle/working/image_sub/best_fold_0.csv \
    /kaggle/working/image_sub/best_fold_1.csv \
    /kaggle/working/image_sub/best_fold_2.csv \
    /kaggle/working/image_sub/best_fold_3.csv \
    /kaggle/working/image_sub/best_fold_4.csv \
    /kaggle/working/ensemble3_image_fold0.csv \
    /kaggle/working/ensemble3_image_fold1.csv \
    /kaggle/working/ensemble3_image_fold2.csv \
    /kaggle/working/ensemble3_image_fold3.csv \
    /kaggle/working/ensemble3_image_fold4.csv \
    -sw 1 \
    -iw 2 2 2 2 2 \
    3 3 3 3 3 \
    2 2 2 2 2 \
    -std2img /kaggle/temp/study2images_d.pickle \
    -img2shape /kaggle/temp/image2shape_d.pickle \
    --iou 0.6 \
    --conf 0.0001

    !rm -rf /kaggle/working/yolov5-develop
    !ls -l /kaggle/working
else:
    pd.read_csv('../input/siim-covid19-detection/sample_submission.csv').to_csv('submission.csv', index = False)